Importing the libraries

In [1]:
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
import os

Data Preprocessing

In [2]:

#Doing data augmantation in order to make the learning more difficult
train_datagen=ImageDataGenerator(
                                   rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2, 
                                   horizontal_flip=True
                                                        )

#preprocessing the training set

training_set= train_datagen.flow_from_directory('training set directory',
                                                target_size=(32,32),
                                                batch_size=32,
                                                class_mode='categorical')

Found 78000 images belonging to 26 classes.


Preprocessing the test set

In [3]:
test_datagen=ImageDataGenerator(rescale=1./255)
test_set=test_datagen.flow_from_directory('test set directory',
                                           target_size=(64,64),
                                           batch_size=32,
                                           class_mode='categorical')


Found 26 images belonging to 26 classes.


Evaluation Set

In [4]:
evaluation_set=ImageDataGenerator(rescale=1./255)
evaluation_set=evaluation_set.flow_from_directory('evaluation set directory',
                                           target_size=(32,32),
                                           batch_size=32,
                                           class_mode='categorical')


Found 1815 images belonging to 26 classes.


Building the cnn

In [5]:
#Initializing the cnn
cnn=tf.keras.models.Sequential()

#adding the first layer
cnn.add(tf.keras.layers.Conv2D(filters=32,
                              kernel_size=3,
                              activation='relu',
                              input_shape=[32,32,3]))
#pooling
cnn.add(tf.keras.layers.MaxPool2D(2,2))

#Second Layer
cnn.add(tf.keras.layers.Conv2D(filters=64,kernel_size=3,padding='same',activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(2,2))

#Third Layer
cnn.add(tf.keras.layers.Conv2D(filters=64,kernel_size=3,padding='same',activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(2,2))



Flattening

In [6]:
#converting the 2-d pixel image matrix into 1-d image matrix for input layer of cnn

cnn.add(tf.keras.layers.Flatten())

Full Connection

In [7]:
#combing all layers of CNN
cnn.add(tf.keras.layers.Dense(units=128,activation='relu'))

Output Layer

In [8]:
#defining the output layer

cnn.add(tf.keras.layers.Dense(units=26,activation='softmax'))

Compiling the CNN

In [9]:
#declairing the optimizer and loss for backprogation process
cnn.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy',tf.keras.metrics.Precision(),tf.keras.metrics.Recall()])

Training the cnn

In [135]:

cnn.fit(x = training_set,validation_data = evaluation_set, epochs = 5)

#Saving the model
cnn.save('my_model.h5') 


Epoch 1/5
2438/2438 [==============================] - 1486s 610ms/step - loss: 0.1107 - accuracy: 0.9614 - precision: 0.9663 - recall: 0.9576 - val_loss: 32.3368 - val_accuracy: 0.1058 - val_precision: 0.1078 - val_recall: 0.1058
Epoch 2/5
2438/2438 [==============================] - 1247s 512ms/step - loss: 0.0948 - accuracy: 0.9673 - precision: 0.9708 - recall: 0.9644 - val_loss: 34.5739 - val_accuracy: 0.1240 - val_precision: 0.1248 - val_recall: 0.1234
Epoch 3/5
2438/2438 [==============================] - 582s 239ms/step - loss: 0.0874 - accuracy: 0.9699 - precision: 0.9725 - recall: 0.9675 - val_loss: 42.0535 - val_accuracy: 0.0909 - val_precision: 0.0923 - val_recall: 0.0909
Epoch 4/5
2438/2438 [==============================] - 172s 71ms/step - loss: 0.0775 - accuracy: 0.9739 - precision: 0.9759 - recall: 0.9720 - val_loss: 41.6831 - val_accuracy: 0.0843 - val_precision: 0.0847 - val_recall: 0.0837
Epoch 5/5
2438/2438 [==============================] - 196s 80ms/step - loss: 0

Knowing Class of each image

In [11]:
class_names=training_set.class_indices

#loading the saved model
n_cnn=tf.keras.models.load_model('my_model.h5')
print(class_names)
d={}
k=0
l=['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']
for i in range(0,26):
    d[i]=l[k]
    k+=1
print(d)


n_cnn.evaluate(evaluation_set)


{'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7, 'I': 8, 'J': 9, 'K': 10, 'L': 11, 'M': 12, 'N': 13, 'O': 14, 'P': 15, 'Q': 16, 'R': 17, 'S': 18, 'T': 19, 'U': 20, 'V': 21, 'W': 22, 'X': 23, 'Y': 24, 'Z': 25}
{0: 'a', 1: 'b', 2: 'c', 3: 'd', 4: 'e', 5: 'f', 6: 'g', 7: 'h', 8: 'i', 9: 'j', 10: 'k', 11: 'l', 12: 'm', 13: 'n', 14: 'o', 15: 'p', 16: 'q', 17: 'r', 18: 's', 19: 't', 20: 'u', 21: 'v', 22: 'w', 23: 'x', 24: 'y', 25: 'z'}
57/57 [==============================] - 5s 78ms/step - loss: 37.8966 - accuracy: 0.0837 - precision: 0.0843 - recall: 0.0832


[37.8966064453125,
 0.08374655991792679,
 0.08431044220924377,
 0.08319558948278427]

Making a single Prediction

In [13]:


import numpy as np
from keras.preprocessing import image

test_image=image.load_img('test_image_directory_for_testing/test.jpg',target_size=(32,32))

#converting image into 2d arrray as cnn takes array as input
test_image=image.img_to_array(test_image)
test_image=np.expand_dims(test_image,axis=0)

result=(n_cnn.predict(test_image))
score = tf.nn.softmax(result[0])
print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(d[np.argmax(score)], 1000 * np.max(score))
)




This image most likely belongs to z with a 98.07 percent confidence.
